In [0]:
df_emp= spark.read.option("nullValue","null").csv('/FileStore/tables/emp.csv', header=True, inferSchema=True)

In [0]:
df_emp.show()

+-----+------+---------+----+----------+----+----+------+------------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+---------+----+----------+----+----+------+------------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|
| 7566| JONES|  MANAGER|7839|1981-02-04|2975|null|    20|  2022-01-05|
| 7654|MARTIN| SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|
| 7698|   SGR|  MANAGER|7839|1981-01-05|2850|null|    30|  2022-01-04|
| 7782|  RAVI|  MANAGER|7839|1981-09-06|2450|null|    10|  2022-01-02|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|  2022-01-02|
| 7839|  KING|PRESIDENT|null|1981-11-01|5000|null|    10|  2022-01-02|
| 7844|TURNER| SALESMAN|7698|1981-08-09|1500|   0|    30|  2022-01-02|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|  2022-01-03|
| 7900

In [0]:
from pyspark.sql.functions import to_date
df_emp=df_emp.withColumn("HIREDATE", to_date("HIREDATE",'dd-MM-yyyy')).fillna({"HIREDATE": '9999-12-31'})
df_emp.show()

+-----+------+---------+----+----------+----+----+------+------------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+---------+----+----------+----+----+------+------------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|
| 7566| JONES|  MANAGER|7839|1981-02-04|2975|null|    20|  2022-01-05|
| 7654|MARTIN| SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|
| 7698|   SGR|  MANAGER|7839|1981-01-05|2850|null|    30|  2022-01-04|
| 7782|  RAVI|  MANAGER|7839|1981-09-06|2450|null|    10|  2022-01-02|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|  2022-01-02|
| 7839|  KING|PRESIDENT|null|1981-11-01|5000|null|    10|  2022-01-02|
| 7844|TURNER| SALESMAN|7698|1981-08-09|1500|   0|    30|  2022-01-02|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|  2022-01-03|
| 7900

In [0]:
from pyspark.sql.functions import date_format
df_emp=df_emp.withColumn("YEAR",date_format("HIREDATE",'yyyy')).withColumn("MONTH",date_format("HIREDATE",'MM'))
df_emp.show()                                                                          

+-----+------+---------+----+----------+----+----+------+------------+----+-----+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|YEAR|MONTH|
+-----+------+---------+----+----------+----+----+------+------------+----+-----+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|1980|   12|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|1981|   02|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|1981|   02|
| 7566| JONES|  MANAGER|7839|1981-02-04|2975|null|    20|  2022-01-05|1981|   02|
| 7654|MARTIN| SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|1981|   09|
| 7698|   SGR|  MANAGER|7839|1981-01-05|2850|null|    30|  2022-01-04|1981|   01|
| 7782|  RAVI|  MANAGER|7839|1981-09-06|2450|null|    10|  2022-01-02|1981|   09|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|  2022-01-02|1987|   04|
| 7839|  KING|PRESIDENT|null|1981-11-01|5000|null|    10|  2022-01-02|1981|   11|
| 7844|TURNER| S

In [0]:
df_emp.write.format("delta").partitionBy("YEAR","MONTH").mode("overwrite").saveAsTable("emp_par")

In [0]:
%fs ls /user/hive/warehouse/emp_par/YEAR=1980/


path,name,size,modificationTime
dbfs:/user/hive/warehouse/emp_par/YEAR=1980/MONTH=12/,MONTH=12/,0,0


In [0]:
%sql
explain select * from emp_par where year = 1980

plan
"== Physical Plan == *(1) ColumnarToRow +- FileScan parquet spark_catalog.default.emp_par[EMPNO#2912,ENAME#2913,JOB#2914,MGR#2915,HIREDATE#2916,SAL#2917,COMM#2918,DEPTNO#2919,UPDATED_DATE#2920,YEAR#2921,MONTH#2922] Batched: true, DataFilters: [], Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[dbfs:/user/hive/warehouse/emp_par], PartitionFilters: [isnotnull(YEAR#2921), (cast(YEAR#2921 as int) = 1980)], PushedFilters: [], ReadSchema: struct<EMPNO:int,ENAME:string,JOB:string,MGR:int,HIREDATE:date,SAL:int,COMM:int,DEPTNO:int,UPDATE..."


In [0]:
### Removing duplicates

In [0]:
dbutils.fs.put("/scenarios/duplicates.csv","""id,name,loc,updated_date
1,ravi,bangalore,2021-01-01
1,ravi,chennai,2022-02-02
1,ravi,Hyderabad,2022-06-10
2,Raj,bangalore,2021-01-01
2,Raj,chennai,2022-02-02
3,Raj,Hyderabad,2022-06-10
4,Prasad,bangalore,2021-01-01
5,Mahesh,chennai,2022-02-02
4,Prasad,Hyderabad,2022-06-10
""")

Wrote 274 bytes.
Out[24]: True

In [0]:
df_dup = spark.read.csv("/scenarios/duplicates.csv",header = True, inferSchema= True)
df_dup.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- updated_date: date (nullable = true)



In [0]:
display(df_dup)

id,name,loc,updated_date
1,ravi,bangalore,2021-01-01
1,ravi,chennai,2022-02-02
1,ravi,Hyderabad,2022-06-10
2,Raj,bangalore,2021-01-01
2,Raj,chennai,2022-02-02
3,Raj,Hyderabad,2022-06-10
4,Prasad,bangalore,2021-01-01
5,Mahesh,chennai,2022-02-02
4,Prasad,Hyderabad,2022-06-10


In [0]:

from pyspark.sql.functions import col
display(df_dup.dropDuplicates(["id"]))

id,name,loc,updated_date,rowid
1,ravi,Hyderabad,2022-06-10,1
2,Raj,chennai,2022-02-02,1
3,Raj,Hyderabad,2022-06-10,1
4,Prasad,Hyderabad,2022-06-10,1
5,Mahesh,chennai,2022-02-02,1


In [0]:
#Window function


from pyspark.sql.window import Window
from pyspark.sql.functions import * 
df_dup=df_dup.withColumn("rowid", row_number().over(Window.partitionBy("id").orderBy(col("updated_date").desc())))

In [0]:
df_uniq = df_dup.filter("rowid = 1")

In [0]:
df_uniq.show()

+---+------+---------+------------+-----+
| id|  name|      loc|updated_date|rowid|
+---+------+---------+------------+-----+
|  1|  ravi|Hyderabad|  2022-06-10|    1|
|  2|   Raj|  chennai|  2022-02-02|    1|
|  3|   Raj|Hyderabad|  2022-06-10|    1|
|  4|Prasad|Hyderabad|  2022-06-10|    1|
|  5|Mahesh|  chennai|  2022-02-02|    1|
+---+------+---------+------------+-----+



In [0]:
#Double Limiter multi limiter

In [0]:
dbutils.fs.put("/schenarios/double_pipe.csv","""id||name||loc
1||ravi||Bangalore
2||Raj||Chennai
3||Mahesh||Hyderabad
4||Prasad||Chennai
5||Sridhar||Pune
""",True)

Wrote 106 bytes.
Out[40]: True

In [0]:
df_del=spark.read.csv('/schenarios/double_pipe.csv', header= True, inferSchema= True, sep ="||")
df_del.show()

+---+-------+---------+
| id|   name|      loc|
+---+-------+---------+
|  1|   ravi|Bangalore|
|  2|    Raj|  Chennai|
|  3| Mahesh|Hyderabad|
|  4| Prasad|  Chennai|
|  5|Sridhar|     Pune|
+---+-------+---------+



In [0]:
dbutils.fs.put("/scenarios/multi_sep.csv","""id,name,loc,marks
1,ravi,Bangalore,35|45|55|65
2,Raj,Chennai,35|45|55|65
3,Mahesh,Hyderabad,35|45|55|65
4,Prasad,Chennai,35|45|55|65
5,Sridhar,Pune,35|45|55|65
""",True)

Wrote 160 bytes.
Out[47]: True

In [0]:
df_mul=spark.read.csv("/scenarios/multi_sep.csv", header= True, inferSchema= True)
df_mul.show()


+---+-------+---------+-----------+
| id|   name|      loc|      marks|
+---+-------+---------+-----------+
|  1|   ravi|Bangalore|35|45|55|65|
|  2|    Raj|  Chennai|35|45|55|65|
|  3| Mahesh|Hyderabad|35|45|55|65|
|  4| Prasad|  Chennai|35|45|55|65|
|  5|Sridhar|     Pune|35|45|55|65|
+---+-------+---------+-----------+



In [0]:
from pyspark.sql.functions import split,col

df_mul=df_mul.withColumn("marks_split", split(col("marks"),"[|]"))\
            .withColumn("SUB1",col("marks_split")[0])\
            .withColumn("SUB2",col("marks_split")[1])\
            .withColumn("SUB3",col("marks_split")[2])\
            .withColumn("SUB4",col("marks_split")[3]).drop("marks_split", "marks")
df_mul.show()

+---+-------+---------+----+----+----+----+
| id|   name|      loc|SUB1|SUB2|SUB3|SUB4|
+---+-------+---------+----+----+----+----+
|  1|   ravi|Bangalore|  35|  45|  55|  65|
|  2|    Raj|  Chennai|  35|  45|  55|  65|
|  3| Mahesh|Hyderabad|  35|  45|  55|  65|
|  4| Prasad|  Chennai|  35|  45|  55|  65|
|  5|Sridhar|     Pune|  35|  45|  55|  65|
+---+-------+---------+----+----+----+----+



In [0]:
### Remove duplicates

In [0]:
from pyspark.sql.functions import * 
df = spark.read.option("nullValue","null").csv("/FileStore/tables/emp.csv",header=True) 
# Take null values count at each column 
null_cols = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]) 
null_cols.show()
# Filter only columns are having null record count.. 
df_out = null_cols.select([key for (key,value) in null_cols.collect()[0].asDict().items() if value> 0])
df_out.show()

+-----+-----+---+---+--------+---+----+------+------------+
|EMPNO|ENAME|JOB|MGR|HIREDATE|SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+-----+---+---+--------+---+----+------+------------+
|    3|    3|  3|  5|       5|  3|  23|     3|           3|
+-----+-----+---+---+--------+---+----+------+------------+

+-----+-----+---+---+--------+---+----+------+------------+
|EMPNO|ENAME|JOB|MGR|HIREDATE|SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+-----+---+---+--------+---+----+------+------------+
|    3|    3|  3|  5|       5|  3|  23|     3|           3|
+-----+-----+---+---+--------+---+----+------+------------+



In [0]:
###Playing with json

dbutils.fs.put("/schenarios/complex_json.json","""[{
	"id": "0001",
	"type": "donut",
	"name": "Cake",
	"ppu": 0.55,
	"batters":
		{
			"batter":
				[
					{ "id": "1001", "type": "Regular" },
					{ "id": "1002", "type": "Chocolate" },
					{ "id": "1003", "type": "Blueberry" },
					{ "id": "1004", "type": "Devil's Food" }
				]
		},
	"topping":
		[
			{ "id": "5001", "type": "None" },
			{ "id": "5002", "type": "Glazed" },
			{ "id": "5005", "type": "Sugar" },
			{ "id": "5007", "type": "Powdered Sugar" },
			{ "id": "5006", "type": "Chocolate with Sprinkles" },
			{ "id": "5003", "type": "Chocolate" },
			{ "id": "5004", "type": "Maple" }
		]
}]""",True)

Wrote 608 bytes.
Out[70]: True

In [0]:
from pyspark.sql.functions import explode,col
df_json = spark.read.option("multiline","true").json("/schenarios/complex_json.json")
df_json.printSchema()
display(df_json.select(explode("batters.batter")))

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



col
"List(1001, Regular)"
"List(1002, Chocolate)"
"List(1003, Blueberry)"
"List(1004, Devil's Food)"


In [0]:
df_final= df_json.withColumn("topping_explode", explode("topping"))\
                 .withColumn("topping_id",col("topping_explode.id"))\
                 .withColumn("topping_type",col("topping_explode.type"))\
                 .drop("topping_explode","topping")\
                 .withColumn("batter_explode",explode("batters.batter"))\
                 .withColumn("batter_id",col("batter_explode.id"))\
                 .withColumn("batter_type",col("batter_explode.type"))\
                 .drop("batters","batter_explode")

display(df_final)

id,name,ppu,type,topping_id,topping_type,batter_id,batter_type
0001,Cake,0.55,donut,5001,None,1001,Regular
0001,Cake,0.55,donut,5001,None,1002,Chocolate
0001,Cake,0.55,donut,5001,None,1003,Blueberry
0001,Cake,0.55,donut,5001,None,1004,Devil's Food
0001,Cake,0.55,donut,5002,Glazed,1001,Regular
0001,Cake,0.55,donut,5002,Glazed,1002,Chocolate
0001,Cake,0.55,donut,5002,Glazed,1003,Blueberry
0001,Cake,0.55,donut,5002,Glazed,1004,Devil's Food
0001,Cake,0.55,donut,5005,Sugar,1001,Regular
0001,Cake,0.55,donut,5005,Sugar,1002,Chocolate


In [0]:
from pyspark.sql.functions import spark_partition_id
df = spark.read.format("delta").load("/user/hive/warehouse/emp_par").repartition(4,"DEPTNO").withColumn("PARTITION_ID",spark_partition_id())
df.createOrReplaceTempView("df")
display(df)

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE,YEAR,MONTH,PARTITION_ID
7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,2022-01-01,1981,02,1
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-01,1981,02,1
7566,JONES,MANAGER,7839,1981-02-04,2975,null,20,2022-01-05,1981,02,1
7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,2022-01-04,1981,02,1
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-04,1981,02,1
7566,JONES,MANAGER,7839,1981-02-04,2975,null,20,2022-01-04,1981,02,1
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-03,1981,09,1
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-05,1981,09,1
7844,TURNER,SALESMAN,7698,1981-08-09,1500,0,30,2022-01-02,1981,08,1
7844,TURNER,SALESMAN,7698,1981-08-09,1500,0,30,2022-01-06,1981,08,1


In [0]:
%sql
select * from df order by sal

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE,YEAR,MONTH,PARTITION_ID
null,null,null,null,9999-12-31,null,null,null,null,9999,12,2
null,null,null,null,9999-12-31,null,null,null,null,9999,12,2
null,null,null,null,9999-12-31,null,null,null,null,9999,12,2
1234,RAM,CLERK,7457,9999-12-31,494,588,80,2022-01-07,9999,12,3
1234,SEKHAR,doctor,7777,9999-12-31,667,78,80,2022-01-03,9999,12,3
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-01,1980,12,1
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-04,1980,12,1
7900,JAMES,CLERK,7698,1981-03-12,950,null,30,2022-01-03,1981,03,1
7900,JAMES,CLERK,7698,1981-03-12,950,null,30,2022-01-07,1981,03,1
7876,ADAMS,CLERK,7788,1987-05-23,1100,null,20,2022-01-03,1987,05,1


In [0]:
%sql
select * from df sort by sal

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE,YEAR,MONTH,PARTITION_ID
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-01,1980,12,1
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-04,1980,12,1
7900,JAMES,CLERK,7698,1981-03-12,950,null,30,2022-01-03,1981,03,1
7900,JAMES,CLERK,7698,1981-03-12,950,null,30,2022-01-07,1981,03,1
7876,ADAMS,CLERK,7788,1987-05-23,1100,null,20,2022-01-03,1987,05,1
7876,ADAMS,CLERK,7788,1987-05-23,1100,null,20,2022-01-06,1987,05,1
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-01,1981,02,1
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-04,1981,02,1
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-03,1981,09,1
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-05,1981,09,1


In [0]:
df.orderBy("SAL").show()


+-----+------+--------+----+----------+----+----+------+------------+----+-----+------------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|YEAR|MONTH|PARTITION_ID|
+-----+------+--------+----+----------+----+----+------+------------+----+-----+------------+
| null|  null|    null|null|9999-12-31|null|null|  null|        null|9999|   12|           2|
| null|  null|    null|null|9999-12-31|null|null|  null|        null|9999|   12|           2|
| null|  null|    null|null|9999-12-31|null|null|  null|        null|9999|   12|           2|
| 1234|   RAM|   CLERK|7457|9999-12-31| 494| 588|    80|  2022-01-07|9999|   12|           3|
| 1234|SEKHAR|  doctor|7777|9999-12-31| 667|  78|    80|  2022-01-03|9999|   12|           3|
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|1980|   12|           1|
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|null|    20|  2022-01-04|1980|   12|           1|
| 7900| JAMES|   CLERK|7698|1981-03-12| 950|null|    30|  20

In [0]:
help(df.orderBy)

Help on method sort in module pyspark.sql.dataframe:

sort(*cols: Union[str, pyspark.sql.column.Column, List[Union[str, pyspark.sql.column.Column]]], **kwargs: Any) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` sorted by the specified column(s).
    
    .. versionadded:: 1.3.0
    
    Parameters
    ----------
    cols : str, list, or :class:`Column`, optional
         list of :class:`Column` or column names to sort by.
    
    Other Parameters
    ----------------
    ascending : bool or list, optional
        boolean or list of boolean (default ``True``).
        Sort ascending vs. descending. Specify list for multiple sort orders.
        If a list is specified, length of the list must equal length of the `cols`.
    
    Examples
    --------
    >>> df.sort(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.sort("age", ascending=False).collect()
    [Row(age=5, name='Bob'), Row(age=

In [0]:
df.sort('SAL').show()

+-----+------+--------+----+----------+----+----+------+------------+----+-----+------------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|YEAR|MONTH|PARTITION_ID|
+-----+------+--------+----+----------+----+----+------+------------+----+-----+------------+
| null|  null|    null|null|9999-12-31|null|null|  null|        null|9999|   12|           2|
| null|  null|    null|null|9999-12-31|null|null|  null|        null|9999|   12|           2|
| null|  null|    null|null|9999-12-31|null|null|  null|        null|9999|   12|           2|
| 1234|   RAM|   CLERK|7457|9999-12-31| 494| 588|    80|  2022-01-07|9999|   12|           3|
| 1234|SEKHAR|  doctor|7777|9999-12-31| 667|  78|    80|  2022-01-03|9999|   12|           3|
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|1980|   12|           1|
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|null|    20|  2022-01-04|1980|   12|           1|
| 7900| JAMES|   CLERK|7698|1981-03-12| 950|null|    30|  20

In [0]:
df.sortWithinPartitions("SAL").show()


+-----+------+--------+----+----------+----+----+------+------------+----+-----+------------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|YEAR|MONTH|PARTITION_ID|
+-----+------+--------+----+----------+----+----+------+------------+----+-----+------------+
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|1980|   12|           1|
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|null|    20|  2022-01-04|1980|   12|           1|
| 7900| JAMES|   CLERK|7698|1981-03-12| 950|null|    30|  2022-01-03|1981|   03|           1|
| 7900| JAMES|   CLERK|7698|1981-03-12| 950|null|    30|  2022-01-07|1981|   03|           1|
| 7876| ADAMS|   CLERK|7788|1987-05-23|1100|null|    20|  2022-01-03|1987|   05|           1|
| 7876| ADAMS|   CLERK|7788|1987-05-23|1100|null|    20|  2022-01-06|1987|   05|           1|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|1981|   02|           1|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|  20

In [0]:
# create new data file using python dynamically.
def createFile(filename):
    f=open("/FileStore/tables/"+filename+".txt","w")
    for i in range(100000):
        f.write("sample data writting : "+str(i))
    f.close()
    print("File Created as : "+str(filename))
    return filename

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
df= spark.read.option("nullValue","null").csv("/FileStore/tables/emp.csv",header=True, inferSchema= True)
df=df.withColumn("ID_KEY",monotonically_increasing_id())
display(df)

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE,ID_KEY
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-01,0
7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,2022-01-01,1
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-01,2
7566,JONES,MANAGER,7839,1981-02-04,2975,null,20,2022-01-05,3
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-03,4
7698,SGR,MANAGER,7839,1981-01-05,2850,null,30,2022-01-04,5
7782,RAVI,MANAGER,7839,1981-09-06,2450,null,10,2022-01-02,6
7788,SCOTT,ANALYST,7566,1987-04-19,3000,null,20,2022-01-02,7
7839,KING,PRESIDENT,null,1981-11-01,5000,null,10,2022-01-02,8
7844,TURNER,SALESMAN,7698,1981-08-09,1500,0,30,2022-01-02,9
